# COVIDvu <img src='resources/UN-flag.png' align='right'>

COVID-19 view various charts.

---
## Runtime prerequisites

In [ ]:
%%capture --no-stderr requirementsOutput

displayRequirementsOutput = False

%pip install -r requirements.txt
from covidvu.utils import autoReloadCode; autoReloadCode()

In [ ]:
if displayRequirementsOutput:
    requirementsOutput.show()

---
## Pull latest datasets from the JH CSSE GitHub repository

This function is `crontab` ready; it can be called from here, the command line, or as a system job.

In [ ]:
%%bash
./refreshdata local

---
## Predict cases for all countries (requires pre-training)

From `./work` please run

```
python covidvu/predict.py all
```

before running these cells

In [ ]:
from covidvu.predict import loadAll
from covidvu.visualize import plotDataAndPredictionsWithCI
from ipywidgets import widgets
from ipywidgets import interact
from ipywidgets import fixed

In [ ]:
confirmedCasesAll, meanPredictionTSAll, percentilesTSAll = loadAll()

In [ ]:
countryNameAll = meanPredictionTSAll.columns

In [ ]:
multiCountry = widgets.SelectMultiple(
    options=countryNameAll,
    value=[countryNameAll[0]],
    description='Country Index',
    disabled=False
)
log = widgets.Checkbox(value=True, description='Log scale')

In [ ]:
interact(plotDataAndPredictionsWithCI,
         meanPredictionTSAll=fixed(meanPredictionTSAll),
         confirmedCasesAll=fixed(confirmedCasesAll),         
         percentilesTSAll=fixed(percentilesTSAll),
         cmapName=fixed('Reds'),
         selectedColumns=multiCountry,         
         log=log
        );

# Long-term forecast

In [ ]:
from covidvu.predict import buildLogisticModel
from covidvu.predict import _dumpRegionPrediction
from covidvu.predict import predictLogisticGrowth
from covidvu.predict import PREDICTIONS_PERCENTILES
from covidvu.predict import PRIOR_GROWTH_RATE
from covidvu.predict import PRIOR_MID_POINT
from covidvu.predict import PRIOR_SIGMA
from covidvu.pipeline.vujson import SITE_DATA
from covidvu.predict import load
import numpy as np
import pandas as pd

In [ ]:
logRegModel = buildLogisticModel((0,np.log10((327.2e6*0.8))),
                                 PRIOR_MID_POINT,
                                 PRIOR_GROWTH_RATE,
                                 PRIOR_SIGMA,
                                )

In [ ]:
prediction = predictLogisticGrowth(logRegModel, countryName='United Kingdom', nDaysPredict=30*2, maxTreedepth=12)
_dumpRegionPrediction(prediction, SITE_DATA, PREDICTIONS_PERCENTILES)

In [ ]:
prediction = predictLogisticGrowth(logRegModel, countryName='US', nDaysPredict=30*2, maxTreedepth=12)
_dumpRegionPrediction(prediction, SITE_DATA, PREDICTIONS_PERCENTILES)

---
&#169; the COVIDvu Contributors.  All rights reserved.